In [1]:
import pandas as pd
import numpy as np

In [2]:
# Define some constants
latitude = [40.79736, 41.78701, 30.1444, 25.7738]
longitude = [-73.97785, -87.77166, -97.66876, -80.1936]
cities = ["ny", "il", "tx", "fl"]
start_date = "2016-01-01"
end_date = "2024-03-12"

In [3]:
# Unpickle the DataFrames
city_history_dfs = []

for i in range(len(cities)):
  city_history_dfs.append(pd.read_pickle("./merged_df_" + cities[i] + ".pkl"))

print("Loaded DFs for " + str(len(city_history_dfs)) + " cities.\n")
print(city_history_dfs[0].info())
print(city_history_dfs[0].loc[0])

Loaded DFs for 4 cities.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 65 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  1447 non-null   object 
 1   tmax_vc               1447 non-null   float64
 2   tmin_vc               1447 non-null   float64
 3   humi_vc               1447 non-null   float64
 4   wind_vc               1447 non-null   float64
 5   tmax_om               1447 non-null   float64
 6   tmin_om               1447 non-null   float64
 7   sund_om               1447 non-null   float64
 8   prec_om               1447 non-null   float64
 9   wind_om               1447 non-null   float64
 10  tavg_ms               1447 non-null   float64
 11  tmin_ms               1447 non-null   float64
 12  tmax_ms               1447 non-null   float64
 13  prcp_ms               1159 non-null   float64
 14  snow_ms               0 non-null      float64


In [4]:
# Figure out what all cols to keep
allCols = city_history_dfs[0].columns.tolist()
print(len(allCols))
print(*allCols, sep = '\n')

65
date
tmax_vc
tmin_vc
humi_vc
wind_vc
tmax_om
tmin_om
sund_om
prec_om
wind_om
tavg_ms
tmin_ms
tmax_ms
prcp_ms
snow_ms
wdir_ms
wspd_ms
wpgt_ms
pres_ms
tsun_ms
station_ncei
name_ncei
latitude_ncei
longitude_ncei
elevation_ncei
awnd_ncei
awnd_attributes_ncei
pgtm_ncei
pgtm_attributes_ncei
prcp_ncei
prcp_attributes_ncei
snow_ncei
snow_attributes_ncei
snwd_ncei
snwd_attributes_ncei
tavg_ncei
tavg_attributes_ncei
tmax_ncei
tmax_attributes_ncei
tmin_ncei
tmin_attributes_ncei
tsun_ncei
tsun_attributes_ncei
wdf2_ncei
wdf2_attributes_ncei
wdf5_ncei
wdf5_attributes_ncei
wsf2_ncei
wsf2_attributes_ncei
wsf5_ncei
wsf5_attributes_ncei
wt01_ncei
wt01_attributes_ncei
wt02_ncei
wt02_attributes_ncei
wt03_ncei
wt03_attributes_ncei
wt04_ncei
wt04_attributes_ncei
wt05_ncei
wt05_attributes_ncei
wt06_ncei
wt06_attributes_ncei
wt08_ncei
wt08_attributes_ncei


In [11]:
ny_data = city_history_dfs[0]

In [27]:
# Keep the columns we need
# date, tmax from all sources, 
# humidity from visual crossing,
# precipitation from open meteo,
# snow from meteo stats
# sunny time from ncei
ny_data = city_history_dfs[0][['date', 'tmax_vc', 'tmax_om', 'tmax_ms', 'tmax_ncei', 'humi_vc', 'prec_om', 'tmin_ms', 'tmin_ncei']]
ny_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       1447 non-null   object 
 1   tmax_vc    1447 non-null   float64
 2   tmax_om    1447 non-null   float64
 3   tmax_ms    1447 non-null   float64
 4   tmax_ncei  1447 non-null   float64
 5   humi_vc    1447 non-null   float64
 6   prec_om    1447 non-null   float64
 7   tmin_ms    1447 non-null   float64
 8   tmin_ncei  1447 non-null   float64
dtypes: float64(8), object(1)
memory usage: 101.9+ KB


In [28]:
ny_data.head()

,date,tmax_vc,tmax_om,tmax_ms,tmax_ncei,humi_vc,prec_om,tmin_ms,tmin_ncei
0,2020-01-15,50.2,9.292999,10.0,10.0,66.4,3.0,6.0,5.0
1,2020-01-16,48.2,8.693000,9.0,9.4,63.7,8.0,1.0,0.6
2,2020-01-17,34.1,2.643000,1.0,0.6,36.0,0.0,-6.0,-6.0
3,2020-01-18,37.1,0.443000,3.0,2.8,60.2,6.0,-7.0,-6.6
4,2020-01-19,42.4,6.793000,6.0,6.1,62.2,7.0,-2.0,-2.7


In [24]:
def cleanAndPreprocessData(df):
  columns_to_convert_to_farhenheit = ['tmax_om', 'tmax_ms', 'tmax_ncei', 'tmin_ms', 'tmin_ncei']
  for column in columns_to_convert_to_farhenheit:
    df[column] = df[column] * 9/5 + 32
  print(df.head())
  df['date'] = pd.to_datetime(df['date']) 
  df['day'] = df['date'].dt.dayofyear
  df['tmax_avg'] = df[['tmax_vc', 'tmax_om', 'tmax_ms', 'tmax_ncei']].mean(axis=1)
  df['tmin_avg'] = df[['tmin_ms', 'tmin_ncei']].mean(axis=1)
  return df

In [29]:
ny_data = cleanAndPreprocessData(ny_data)

         date  tmax_vc    tmax_om  tmax_ms  tmax_ncei  humi_vc  prec_om  \
0  2020-01-15     50.2  48.727398     50.0      50.00     66.4      3.0   
1  2020-01-16     48.2  47.647400     48.2      48.92     63.7      8.0   
2  2020-01-17     34.1  36.757400     33.8      33.08     36.0      0.0   
3  2020-01-18     37.1  32.797400     37.4      37.04     60.2      6.0   
4  2020-01-19     42.4  44.227400     42.8      42.98     62.2      7.0   

   tmin_ms  tmin_ncei  
0     42.8      41.00  
1     33.8      33.08  
2     21.2      21.20  
3     19.4      20.12  
4     28.4      27.14  


/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_64976/1328700341.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column] * 9/5 + 32
/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_64976/1328700341.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_64976/1328700341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [30]:
ny_data

,date,tmax_vc,tmax_om,tmax_ms,tmax_ncei,humi_vc,prec_om,tmin_ms,tmin_ncei,day,tmax_avg,tmin_avg
0,2020-01-15,50.2,48.727398,50.00,50.00,66.4,3.0,42.80,41.00,15,49.73185,41.90
1,2020-01-16,48.2,47.647400,48.20,48.92,63.7,8.0,33.80,33.08,16,48.24185,33.44
2,2020-01-17,34.1,36.757400,33.80,33.08,36.0,0.0,21.20,21.20,17,34.43435,21.20
3,2020-01-18,37.1,32.797400,37.40,37.04,60.2,6.0,19.40,20.12,18,36.08435,19.76
4,2020-01-19,42.4,44.227400,42.80,42.98,62.2,7.0,28.40,27.14,19,43.10185,27.77
...,...,...,...,...,...,...,...,...,...,...,...,...
1442,2024-03-04,57.7,58.267398,61.70,59.00,71.0,0.0,46.04,46.04,64,59.16685,46.04
1443,2024-03-05,47.9,46.747400,48.02,48.92,87.6,16.0,42.80,42.08,65,47.89685,42.44
1444,2024-03-06,52.6,49.987400,52.70,53.06,86.1,6.0,42.26,46.94,66,52.08685,44.60
1445,2024-03-07,53.9,54.757400,51.62,53.96,73.9,13.0,41.72,44.06,67,53.55935,42.89


In [31]:
ny_data.to_pickle("./ny_data_cleaned.pkl")  